Notes:
1. Velocity is set to a value of [0,15] for p1, p2, and NO. Triangle has velocity on and off (0 or 1)
2. All tracks seem to be in 4/4 timing (I checked this by printing "time_signature_changes". The count for numerator=4, denominator=4 is 2439, which is the same number of tracks with 4 instruments)

In [ ]:
#!pip install nesmdb==0.1.8
#!pip install pretty_midi
#!wget http://deepyeti.ucsd.edu/cdonahue/nesmdb/nesmdb_midi.tar.gz
#!tar xvfz nesmdb_midi.tar.gz

In [25]:
import numpy as np
import note_seq

In [88]:
# This portion of code gets a list of all files with 4 instruments that are not corrupted
import os
import pretty_midi

four_instruments = []

for files in os.walk('nesmdb_midi/train'):
    for file in files[2]:
        if file.endswith('.mid'):
            #print(file)
            try:
                midi_data = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + file)
                if len(midi_data.instruments) > 3:
                    four_instruments.append(file)
                    print(file)
                    print(midi_data.instruments)
                    print(midi_data.time_signature_changes)
            except:
                print("error on file: " + file)
            

108_Famista_90_10_11Unknown2.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
[TimeSignature(numerator=4, denominator=4, time=0.0), TimeSignature(numerator=1, denominator=1, time=25.658798185941045)]
178_Ironsword_Wizards_amp_WarriorsII_12_13FireElemental.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
[TimeSignature(numerator=4, denominator=4, time=0.0), TimeSignature(numerator=1, denominator=1, time=24.426462585034013)]
195_KonamiWaiWaiWorld_18_19FinalStageBossBGM.mid
[Instrument(program=80, is_drum=False, name="p1"), Instrument(program=81, is_drum=False, name="p2"), Instrument(program=38, is_drum=False, name="tr"), Instrument(program=121, is_drum=True, name="no")]
[TimeSignat

In [103]:
len(four_instruments)

2439

In [97]:
four_instruments[0]

'108_Famista_90_10_11Unknown2.mid'

In [101]:
import os

#We should create a new folder for Train, Test, and Validate

#Create separate midi files for each instrument
#Input: midi file with 4 instruments, name of directory to save new files
#Output: 4 midi files, each containing one instrument
#Creates pretty midi object for each instrument
#Appends the instrument to the pretty midi object
#Writes the pretty midi object to a file
def create_separate_midi_tracks(midi_file, new_dir):

    os.makedirs(new_dir, exist_ok=True)

    pm = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + midi_file)
    midi_file_stripped = midi_file.split('.mid')[0]
    #Pulse 1
    track_1_instrument_1 = pretty_midi.PrettyMIDI()
    track_1_instrument_1.instruments.append(pm.instruments[0])
    track_1_instrument_1.write(new_dir + '/' + midi_file_stripped + '_inst_1.mid')
    #Pulse 2
    track_1_instrument_2 = pretty_midi.PrettyMIDI()
    track_1_instrument_2.instruments.append(pm.instruments[1])
    track_1_instrument_2.write(new_dir + '/' + midi_file_stripped + '_inst_2.mid')
    #Triangle
    track_1_instrument_3 = pretty_midi.PrettyMIDI()
    track_1_instrument_3.instruments.append(pm.instruments[2])
    track_1_instrument_3.write(new_dir + '/' + midi_file_stripped + '_inst_3.mid')
    #Noise
    track_1_instrument_4 = pretty_midi.PrettyMIDI()
    track_1_instrument_4.instruments.append(pm.instruments[3])
    track_1_instrument_4.write(new_dir + '/' + midi_file_stripped + '_inst_4.mid')

#Loads the individual instrument midi tracks into pretty midi objects
#Input: midi_file (name of the midi files), dir (directory where instrument midi files are saved)
#Output: Four instrument midi files - Order is P1, P2, Tri, NO
def load_separate_midi_tracks(midi_file, dir):
    #Pulse 1
    midi_instrument_1 = pretty_midi.PrettyMIDI(dir + '/' + midi_file + '_inst_1.mid')
    #Pulse 2
    midi_instrument_2 = pretty_midi.PrettyMIDI(dir + '/' + midi_file + '_inst_2.mid')
    #Triangle
    midi_instrument_3 = pretty_midi.PrettyMIDI(dir + '/' + midi_file + '_inst_3.mid')
    #Noise
    midi_instrument_4 = pretty_midi.PrettyMIDI(dir + '/' + midi_file + '_inst_4.mid')

    return midi_instrument_1, midi_instrument_2, midi_instrument_3, midi_instrument_4

# ----------------------------------------------------------------
# -----------           Load from Midi             --------------
# ----------------------------------------------------------------


from hvo_sequence import midi_to_hvo_sequence

#Create Mapping for the MIDI values in the tracks. Since these are pitched, we make a mapping from [0, 127]
mapping_array = list(range(0,128))
PITCH_MAPPING = {
    'Instrument': mapping_array
}

# Create 4 HVO Sequences for each individual MIDI instrument file

def load_hvo_from_midi(mapping, midi_dir, midi_file):
    hs_individual_streams_list = []

    max_length = 0
    #Load the four midi instrument files into HVO Sequences
    for i in range(4):
        hs_individual_streams_list.append(
            midi_to_hvo_sequence(
                filename= midi_dir + '/' + midi_file + '_inst_' + str(i+1) + '.mid',
                drum_mapping=mapping,
                beat_division_factors=[4]))
        
        #Find the longest HVO Sequence
        max_length = max(max_length, hs_individual_streams_list[i].hvo.shape[0])

    #Adjust all HVO Sequences so they have the same length as the longest
    for i in range(4):
        hs_individual_streams_list[i].adjust_length(max_length)

    return hs_individual_streams_list

#Compile all four individual voice HVO sequences into a single HVO sequence
#Input: mapping (drum mapping), hs_individual_streams_list (list of four individual instrument streams)
#Output: HVO sequence with four instruments
from hvo_sequence import HVO_Sequence
def combine_individual_instrument_hvo_to_single_hvo(mapping, hs_individual_streams_list):

    #We set each voice to a Midi Mapping value for Drums
    #Four_Voices = {
    #    "voice_1": [36],
    #    "voice_2": [37],
    #    "voice_3": [38],
    #    "voice_4": [39]
    #}

    #Create new HVO Sequence with beat division of 4 and set Drum Mapping to Four_Voices Mapping
    hvo_seq_all_voices = HVO_Sequence(
        beat_division_factors=[4],
        drum_mapping=mapping)

    #Add tempo of 120 BPM to the HVO sequence at time 0
    hvo_seq_all_voices.add_tempo(
        time_step=0,
        qpm=120
    )

    #Add a time signature of 4/4 for the HVO Sequence
    hvo_seq_all_voices.add_time_signature(
        time_step=0,
        numerator=4,
        denominator=4
    )

    #Adjust the length of the HVO Sequence to 
    hvo_seq_all_voices.adjust_length(hs_individual_streams_list[0].hvo.shape[0])

    #print (hvo_seq_all_voices.hvo.shape)

    for i in range(4):
        hvo_seq_all_voices.hvo[:, i] = hs_individual_streams_list[i].hvo[:, 0]
        hvo_seq_all_voices.hvo[:, i+4] = hs_individual_streams_list[i].hvo[:, 1]
        hvo_seq_all_voices.hvo[:, i+8] = hs_individual_streams_list[i].hvo[:, 2]

    return hvo_seq_all_voices


In [102]:
#Going through list of midi files with four voices and creating four separate midi files for each in new directory "nesmbd_midi_four_instruments_train"
for midi in four_instruments:
    create_separate_midi_tracks(midi, 'nesmdb_midi_four_instruments/train')

In [ ]:
#Gives Piano Roll Representation of HVO Sequence
from bokeh.io import output_notebook, show
output_notebook()
hvo_seq_all_voices.to_html_plot(
    filename="DEMO_PATTERN_test_1.html",
    save_figure=False,
    show_figure=True)

In [ ]:
#Create Mapping for the MIDI values in the tracks. Since these are pitched, we make a mapping from [0, 127]
#mapping_array = list(range(0,128))
#TEST_MAPPING = {
#    'Instrument': mapping_array
#}

# ----------------------------------------------------------------
# -----------           Load from Midi             --------------
# ----------------------------------------------------------------

# For each of the 4 instument midi tracks we made, we will create HVO sequences for each of them
# We will find the longest HVO sequence and adjust all other to have this length
# from hvo_sequence import midi_to_hvo_sequence
# hs_individual_streams_list = []

# max_length = 0

# for i in range(4):
#     hs_individual_streams_list.append(
#         midi_to_hvo_sequence(
#             filename='nesmdb_midi/train/' + four_instruments[100] + '_inst_' + str(i+1) + '.mid',
#             drum_mapping=TEST_MAPPING,
#             beat_division_factors=[4]))
#     max_length = max(max_length, hs_individual_streams_list[i].hvo.shape[0])

# for i in range(4):
#     hs_individual_streams_list[i].adjust_length(max_length)

# hs_individual_streams_list

In [ ]:
# #Compile all four individual voice HVO sequences into a single HVO sequence
# from hvo_sequence import HVO_Sequence

# #We set each voice to a Midi Mapping value for Drums
# Four_Voices = {
#     "voice_1": [36],
#     "voice_2": [37],
#     "voice_3": [38],
#     "voice_4": [39]
# }

# #Create new HVO Sequence with beat division of 4 and set Drum Mapping to Four_Voices Mapping
# hvo_seq_all_voices = HVO_Sequence(
#     beat_division_factors=[4],
#     drum_mapping=Four_Voices)

# #Add tempo of 120 BPM to the HVO sequence at time 0
# hvo_seq_all_voices.add_tempo(
#     time_step=0,
#     qpm=120
# )

# #Add a time signature of 4/4 for the HVO Sequence
# hvo_seq_all_voices.add_time_signature(
#     time_step=0,
#     numerator=4,
#     denominator=4
# )

# #Adjust the length of the HVO Sequence to 
# hvo_seq_all_voices.adjust_length(hs_individual_streams_list[0].hvo.shape[0])

# print (hvo_seq_all_voices.hvo.shape)

# for i in range(4):
#     hvo_seq_all_voices.hvo[:, i] = hs_individual_streams_list[i].hvo[:, 0]
#     hvo_seq_all_voices.hvo[:, i+4] = hs_individual_streams_list[i].hvo[:, 1]
#     hvo_seq_all_voices.hvo[:, i+8] = hs_individual_streams_list[i].hvo[:, 2]

In [ ]:
# #Create a separate midi file for each instrument

# track_1_inst_1 = pretty_midi.PrettyMIDI()
# track_1_inst_1.instruments.append(pm.instruments[0])
# track_1_inst_1.write('nesmdb_midi/train/' + four_instruments[10] + '_inst_1.mid')

# track_1_inst_2 = pretty_midi.PrettyMIDI()
# track_1_inst_2.instruments.append(pm.instruments[1])
# track_1_inst_2.write('nesmdb_midi/train/' + four_instruments[100] + '_inst_2.mid')

# track_1_inst_3 = pretty_midi.PrettyMIDI()
# track_1_inst_3.instruments.append(pm.instruments[2])
# track_1_inst_3.write('nesmdb_midi/train/' + four_instruments[100] + '_inst_3.mid')

# track_1_inst_4 = pretty_midi.PrettyMIDI()
# track_1_inst_4.instruments.append(pm.instruments[3])
# track_1_inst_4.write('nesmdb_midi/train/' + four_instruments[100] + '_inst_4.mid')

In [ ]:
# #Load all the files back into pretty_midi

# pm_test1 = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + four_instruments[10] + '_inst_1.mid')
# pm_test2 = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + four_instruments[10] + '_inst_2.mid')
# pm_test3 = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + four_instruments[10] + '_inst_3.mid')
# pm_test4 = pretty_midi.PrettyMIDI('nesmdb_midi/train/' + four_instruments[10] + '_inst_4.mid')

In [ ]:
# #Gives Piano Roll Representation of HVO Sequence
# from bokeh.io import output_notebook, show
# output_notebook()
# hvo_seq_all_voices.to_html_plot(
#     filename="DEMO_PATTERN_test_1.html",
#     save_figure=False,
#     show_figure=True)

Loading BokehJS ...

figure(id='p1622', ...)

In [ ]:

# for tempos in hvo_seq_test_1.tempos:
#     hvo_seq_all_voices.add_tempo(tempos[0], tempos[1])

# for time_sigs in hvo_seq_test_1.time_signatures:
#     hvo_seq_all_voices.add_time_signature(time_sigs[0], time_sigs[1], time_sigs[2])

    